In [4]:
!pip install../ input / scikit-learn-10dev0 / scikit_learn-1.0.dev0-cp37-cp37m-manylinux2010_x86_64.whl
# !pip install -q ../input/resnest-v0-0-5/resnest-0.0.5-py3-none-any.whl
!pip install -q "../input/resnest50-fast-package/resnest-0.0.6b20200701/resnest"
BEST_THRESHOLD = 0.100428
BEST_NOCALL_THRESHOLD = 0.169353
import warnings

warnings.filterwarnings('ignore')

In [5]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

# sys.path.append("../input/birdclef-toolkit-v0530-1930/lib")
sys.path.append("../input/kyle-namakemono-lib/kyle_namakemono_lib")
import bird_recognition

DEVICE: cuda


In [7]:
filepath_list = [
    # "../input/50-fold0-epoch-52-f1-val-05491-20220204015555/train_soundscape_labels_probabilitiy_birdclef_resnest50_fold0_epoch_52_f1_val_05491_20220204015555.csv", # spacer
    #  "../input/50-script-predict-species/birdclef_resnest26d_fold0_epoch_00_f1_val_04130_20211126141934.csv" # baseline full, very bad
    "../input/my-classic-csv/birdclef_resnest26d_fold0_epoch_59_f1_val_03803_20210529151240.csv"  # victory
    # "../input/birdclef-groupby-author-05221040-728258/birdclef_resnest26d_fold0_epoch_59_f1_val_03803_20210529151240.csv", # victory, won private, lost public
    # "../input/50-script-predict-species/birdclef_resnest26d_fold3_epoch_59_f1_val_04684_20210529111322.csv", # breeze, lost private, won public
    # "../input/50-script-predict-species/birdclef_resnest50_fold4_epoch_14_f1_val_04498_20210522015642.csv", # engineer, won private, lost public
    # "../input/50-script-predict-species/birdclef_resnest50_fold4_epoch_31_f1_val_04919_20210522031027.csv", # elbow, won private, lost public
    # "../input/birdclef-resnest26d-fold4-epoch-56-f1-val-04892/birdclef_resnest26d_fold4_epoch_56_f1_val_04892_20210530120940.csv", # generated, won private, lost public
    # "../input/metadata-probability-v0525-2100/birdclef_resnest26d_fold1_epoch_56_f1_val_04461_20210530073109.csv", # given, checked
    # "../input/metadata-probability-v0525-2100/birdclef_resnest26d_fold2_epoch_72_f1_val_04707_20210530125330.csv", # given, checked
    # "../input/metadata-probability-v0525-2100/birdclef_resnest50_fold0_epoch_33_f1_val_03859_20210524151554.csv",
    # "../input/metadata-probability-v0525-2100/birdclef_resnest50_fold1_epoch_34_f1_val_04757_20210524185455.csv", # 👑fold: 1
    # "../input/metadata-probability-v0525-2100/birdclef_resnest50_fold2_epoch_34_f1_val_05027_20210524223209.csv",
    # "../input/metadata-probability-v0525-2100/birdclef_resnest50_fold3_epoch_20_f1_val_04299_20210525010703.csv",
    # "../input/metadata-probability-v0525-2100/birdclef_resnest50_fold4_epoch_34_f1_val_05140_20210525074929.csv"  
]
prob_df = pd.concat([pd.read_csv(_) for _ in filepath_list])

In [8]:
# "../input/bird-probabilities/train_bird_call_prob_{0,1,2}.csv" # CV: 0.7764
class TrainingConfig:
    def __init__(self, debug: bool):
        # MEMO: Configのnocallの挿入時の閾値とは違うので注意！
        self.nocall_threshold: float = 0.5

        self.debug = debug
        self.num_kfolds: int = 5
        self.num_spieces: int = 397
        self.num_candidates: int = 5
        self.max_distance: int = 15  # 20
        self.weight_rate: float = 1.0
        self.sampling_strategy: float = None  # 1.0
        self.random_state: int = 777
        self.num_prob: int = 6
        self.min_rating = None  # choose from  [1,2,3,4,5, None]
        self.use_to_birds = True
        self.use_add_secondlabel = False  # True
        self.xgb_params = {
            "objective": "binary:logistic",
            "tree_method": 'gpu_hist',
            "n_estimators": 1000,
        }
        self.lgb_params = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'device': 'gpu',
        }
        self.weights_filepath_dict = {
            # 'xgb':[f"./xgb_{kfold_index}.pkl" for kfold_index in range(self.num_kfolds)],
            'lgbm': [f"./lgbm_{kfold_index}.pkl" for kfold_index in range(self.num_kfolds)],
            # 'cat':[f"./cat_{kfold_index}.pkl" for kfold_index in range(self.num_kfolds)]
        }


training_config = TrainingConfig(
    debug=False
)
if training_config.debug:
    prob_df = prob_df.head(1000)

In [9]:
class Config:
    def __init__(self):
        self.num_kfolds: int = training_config.num_kfolds
        self.num_spieces: int = training_config.num_spieces
        self.num_candidates: int = training_config.num_candidates
        self.max_distance: int = training_config.max_distance
        self.nocall_threshold: float = training_config.nocall_threshold
        self.num_prob: int = training_config.num_prob
        # Whether to check the score when cutting at the optimum threshold
        self.check_baseline: bool = True
        # A list of model file paths to use when deciding whether the bird is okay
        self.weights_filepath_dict = training_config.weights_filepath_dict
        # Threshold for nocall insertion
        self.nocall_threshold = BEST_NOCALL_THRESHOLD
        # Model weights that predict the probability of each bird singing per frame
        self.checkpoint_paths = [
            # 1
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=F-f=F/birdclef_resnest50_fold0_epoch_69_f1_val_05432_20220306021412.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=F-f=F/birdclef_resnest50_fold0_epoch_69_f1_val_05432_20220315050901.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=F-f=F/birdclef_resnest50_fold0_epoch_76_f1_val_05472_20220306025541.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=F-f=F/birdclef_resnest50_fold0_epoch_76_f1_val_05472_20220315055231.pth'),
            # 2
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=F-f=T/birdclef_resnest50_fold0_epoch_64_f1_val_05390_20220331203014.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=F-f=T/birdclef_resnest50_fold0_epoch_70_f1_val_05404_20220331211250.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=F-f=T/birdclef_resnest50_fold0_epoch_73_f1_val_05393_20220331213408.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=F-f=T/birdclef_resnest50_fold0_epoch_98_f1_val_05447_20220401003156.pth'),
            # 3
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=T-f=T/birdclef_resnest50_fold0_epoch_59_f1_val_05442_20220401075057.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=T-f=T/birdclef_resnest50_fold0_epoch_66_f1_val_05458_20220401084110.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=T-f=T/birdclef_resnest50_fold0_epoch_73_f1_val_05442_20220401093127.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=F-v=T-f=T/birdclef_resnest50_fold0_epoch_98_f1_val_05451_20220401123125.pth'),
            # 4
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=T-v=F-f=F/birdclef_resnest50_fold0_epoch_31_f1_val_03876_20220315112956.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=T-v=F-f=F/birdclef_resnest50_fold0_epoch_69_f1_val_03920_20220315151217.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=T-v=F-f=F/birdclef_resnest50_fold0_epoch_73_f1_val_03850_20220315153541.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=T-v=F-f=F/birdclef_resnest50_fold0_epoch_97_f1_val_03874_20220315175725.pth'),
            # 5
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=T-v=T-f=T/birdclef_resnest50_fold0_epoch_73_f1_val_03932_20220316022151.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=T-v=T-f=T/birdclef_resnest50_fold0_epoch_84_f1_val_03891_20220316033559.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=T-v=T-f=T/birdclef_resnest50_fold0_epoch_90_f1_val_03901_20220316041601.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=0.5-gba=T-v=T-f=T/birdclef_resnest50_fold0_epoch_99_f1_val_03917_20220316051552.pth'),
            # 6
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=F-v=F-f=F/birdclef_resnest50_fold0_epoch_57_f1_val_05510_20220304193608.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=F-v=F-f=F/birdclef_resnest50_fold0_epoch_87_f1_val_05507_20220304222736.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=F-v=F-f=F/birdclef_resnest50_fold0_epoch_92_f1_val_05554_20220304225616.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=F-v=F-f=F/birdclef_resnest50_fold0_epoch_95_f1_val_05507_20220304231323.pth'),
            # 7
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=F-v=T-f=T/birdclef_resnest50_fold0_epoch_72_f1_val_05574_20220322112558.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=F-v=T-f=T/birdclef_resnest50_fold0_epoch_73_f1_val_05586_20220322113215.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=F-v=T-f=T/birdclef_resnest50_fold0_epoch_74_f1_val_05608_20220322113834.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=F-v=T-f=T/birdclef_resnest50_fold0_epoch_97_f1_val_05568_20220322140525.pth'),
            # 8
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=F-f=F/birdclef_resnest50_fold0_epoch_67_f1_val_03964_20220305055936.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=F-f=F/birdclef_resnest50_fold0_epoch_79_f1_val_03946_20220305070659.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=F-f=F/birdclef_resnest50_fold0_epoch_97_f1_val_03962_20220305084800.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=F-f=F/birdclef_resnest50_fold0_epoch_99_f1_val_03946_20220305085915.pth'),
            # 9
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=T-f=F/birdclef_resnest50_fold0_epoch_74_f1_val_03948_20220313061446.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=T-f=F/birdclef_resnest50_fold0_epoch_75_f1_val_03974_20220313062101.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=T-f=F/birdclef_resnest50_fold0_epoch_82_f1_val_03981_20220313070443.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=T-f=F/birdclef_resnest50_fold0_epoch_93_f1_val_04007_20220313081316.pth'),
            # 10
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=T-f=T/birdclef_resnest50_fold0_epoch_70_f1_val_04027_20220305162028.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=T-f=T/birdclef_resnest50_fold0_epoch_71_f1_val_04105_20220305162639.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=T-f=T/birdclef_resnest50_fold0_epoch_84_f1_val_04050_20220305174702.pth'),
            Path(
                '../input/highdiversitymodels/nc/4-nc-id=xx-ma=5.0-gba=T-v=T-f=T/birdclef_resnest50_fold0_epoch_94_f1_val_04060_20220305184913.pth'),
        ]
        self.pred_filepath_list = [
            self.get_prob_filepath_from_checkpoint(path) for path in self.checkpoint_paths
        ]
        self.threshold = BEST_THRESHOLD

    def get_prob_filepath_from_checkpoint(self, checkpoint_path: Path) -> str:
        filename = f"train_soundscape_labels_probabilitiy_%s.csv" % checkpoint_path.stem
        return filename


config = Config()

In [10]:
% % time
submission_df = bird_recognition.evaluation.run(
    training_config,
    config,
    prob_df,
    model_dict=config.weights_filepath_dict,
)

[exclude other]before: 202986
[exclude other]after: 136431
Candidate nocall ratio: 0.1032


  0%|          | 0/20 [00:00<?, ?it/s]

Save to train_soundscape_labels_probabilitiy_birdclef_resnest50_fold0_epoch_69_f1_val_05432_20220306021412.csv


  0%|          | 0/20 [00:00<?, ?it/s]

Save to train_soundscape_labels_probabilitiy_birdclef_resnest50_fold0_epoch_69_f1_val_05432_20220315050901.csv


...

Save to train_soundscape_labels_probabilitiy_birdclef_resnest50_fold0_epoch_71_f1_val_04105_20220305162639.csv


  0%|          | 0/20 [00:00<?, ?it/s]

Save to train_soundscape_labels_probabilitiy_birdclef_resnest50_fold0_epoch_84_f1_val_04050_20220305174702.csv


  0%|          | 0/20 [00:00<?, ?it/s]

Save to train_soundscape_labels_probabilitiy_birdclef_resnest50_fold0_epoch_94_f1_val_04060_20220305184913.csv
training of lgbm is going...
features ['year', 'month', 'sum_prob', 'mean_prob', 'max_prob', 'prev6_prob', 'prev5_prob', 'prev4_prob', 'prev3_prob', 'prev2_prob', 'prev_prob', 'prob', 'next_prob', 'next2_prob', 'next3_prob', 'next4_prob', 'next5_prob', 'next6_prob', 'rank', 'latitude', 'longitude', 'bird_id', 'seconds', 'num_appear', 'site_num_appear', 'site_appear_ratio', 'prob_diff']
fold 0
[LightGBM] [Info] Number of positive: 149206, number of negative: 396529
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5512
[LightGBM] [Info] Number of data points in the train set: 545735, number of used features: 27
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGB

In [11]:
submission_df.to_csv("submission.csv", index=False)